This notebook is used to score the data using the chosen model.
This contains the code of proper way of using ohe.
Train and save model here.

11-11-2024 </br>
checking what went wrong; why I saved lgbm algorithm for high cibil segment instead of the
random forest algorithm. </br>
Result: issue with saving pipeline with 

In [1]:
#installing dependencies
#pip install pandas
#pip install scipy
#pip install -U scikit-learn
#pip install catboost
#pip install xgboost
#pip install lightgbm

In [9]:
import pandas as pd
import numpy as np
import pickle

In [2]:
import sys
print(sys.version)

import numpy as np
print(np.__version__)

import scipy
print(scipy.__version__)

import sklearn
print(sklearn.__version__)

import catboost
print(catboost.__version__)

import xgboost
print(xgboost.__version__)

import lightgbm as lgb
print(lgb.__version__)

3.12.2 (tags/v3.12.2:6abddd9, Feb  6 2024, 21:26:36) [MSC v.1937 64 bit (AMD64)]
1.26.4
1.15.1
1.5.0
1.2.7
2.1.3
4.5.0


In [3]:
# #load wrangled_data
# segment = 'NTC'
# X = pd.read_csv(f'./Wrangled_Data/wrangled_{segment}.csv')
# #CB_model_auc(X, cbc_high_CIBIL_vf, segment)

cat_features = ['product_description', 'marital_status', 'occupation_type','industry_grp',
                'res_city_grp','res_state_grp','res_resident_status_grp','branch_grp',
                'manufacturer_grp', 'gender','nature_of_business','channel_grp','asset_model_grp']

# NTC Scoring

In [4]:
def cb_model_builder2( depth = 6,
                     learning_rate = 0.03,
                     iterations =1000,
                     l2_leaf_reg = 3.0,
                     border_count = 50,
                     bagging_temperature = 1,
                     random_strength = 1,
                     od_wait = 10,
                     auto_class_weights = "Balanced",
                     random_state = 42,
                     objective = 'Logloss',
                     cat_features = cat_features
                    ):
    
    from catboost import CatBoostClassifier
    #catboost implementation
    model = CatBoostClassifier(depth = depth,
                     learning_rate = learning_rate,
                     iterations =iterations,
                     l2_leaf_reg = l2_leaf_reg,
                     border_count = border_count,
                     bagging_temperature = bagging_temperature,
                     random_strength = random_strength,
                     od_wait = od_wait,
                     auto_class_weights = auto_class_weights,
                     random_state = random_state,
                     objective = objective,
                     cat_features = cat_features)
    
    return model

In [5]:
#NTC_level: tuned
cbc_ntc_vf = cb_model_builder2( depth = 4,
                     learning_rate = 0.0979722306417506,
                     iterations =319,
                     l2_leaf_reg = 10.909079937846313,
                     border_count = 233,
                     bagging_temperature = 1.349262400109297,
                     random_strength = 1.959828624191452,
                     od_wait = 87,
                     auto_class_weights = "Balanced",
                     random_state = 42,
                     objective = 'Logloss',
                     cat_features = cat_features
                    )

# Training model

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns

from sklearn.model_selection import (train_test_split, GridSearchCV)
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier, Pool

#load wrangled_data
segment = 'NTC'
X = pd.read_csv(f'./Wrangled_Data/wrangled_{segment}.csv')

cleaned_df = X.copy()

y = 'D3_within12months'
X = cleaned_df.drop(columns = [y, "loan_account_no"])
y = cleaned_df[y]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 11, train_size = 0.8,
                                                    test_size = 0.2, stratify = y)

cat_features = X_train.select_dtypes(include=['object','category']).columns.tolist()

cb_model = cbc_ntc_vf

cb_model.fit(X_train, y_train)

0:	learn: 0.6881232	total: 213ms	remaining: 1m 7s
1:	learn: 0.6816933	total: 273ms	remaining: 43.3s
2:	learn: 0.6762746	total: 330ms	remaining: 34.7s
3:	learn: 0.6718581	total: 371ms	remaining: 29.3s
4:	learn: 0.6674340	total: 421ms	remaining: 26.5s
5:	learn: 0.6641861	total: 477ms	remaining: 24.9s
6:	learn: 0.6607948	total: 530ms	remaining: 23.6s
7:	learn: 0.6586520	total: 564ms	remaining: 21.9s
8:	learn: 0.6555933	total: 606ms	remaining: 20.9s
9:	learn: 0.6540603	total: 659ms	remaining: 20.4s
10:	learn: 0.6527552	total: 687ms	remaining: 19.2s
11:	learn: 0.6513924	total: 742ms	remaining: 19s
12:	learn: 0.6491132	total: 792ms	remaining: 18.6s
13:	learn: 0.6480053	total: 832ms	remaining: 18.1s
14:	learn: 0.6473867	total: 902ms	remaining: 18.3s
15:	learn: 0.6458539	total: 965ms	remaining: 18.3s
16:	learn: 0.6450676	total: 1.01s	remaining: 17.9s
17:	learn: 0.6447072	total: 1.03s	remaining: 17.2s
18:	learn: 0.6433383	total: 1.07s	remaining: 16.9s
19:	learn: 0.6417896	total: 1.11s	remaining

In [15]:
import pickle
#cb_model.save_model('cb_model_NTC.cbm')
#pickle.dump(cb_model, open(f'./Refreshed_models/cb_modelr_NTC.pkl', 'wb'))

In [10]:
#temp
pickle.dump(cb_model, open(f'cb_modelr_NTC.pkl', 'wb'))

In [7]:
cb_model

In [13]:
X.columns

Index(['down_payment_amount', 'disbursal_amount', 'requested_loan_amount',
       'loan_asset_cost', 'final_sanctioned_amount', 'original_emi_amount',
       'age', 'number_of_installments', 'total_income', 'product_description',
       'marital_status', 'occupation_type', 'monthly_income', 'dp_ratio',
       'industry_grp', 'res_city_grp', 'res_state_grp',
       'res_resident_status_grp', 'branch_grp', 'manufacturer_grp'],
      dtype='object')

In [16]:
len(X.columns)

20

In [19]:
y_pred_proba = cb_model.predict_proba(X)[:,1]

cleaned_df['probability'] = y_pred_proba
cleaned_df_NTC_indexed = cleaned_df.copy()

# low_CIBIL Scoring

In [21]:
#tuning lgb jied
def lgbc_model_builder3(boosting_type = 'gbdt',
        max_depth = -1,
        learning_rate = 0.1,
        min_split_gain = 0.0,
        num_leaves = 31,
        n_estimators = 100,
        min_child_samples = 50,
        subsample = 1,
        reg_alpha = 0,
        reg_lambda = 0,
        random_state = 42,
        class_weight = 'balanced',
        n_jobs = -1,
        objective = 'binary'):
    
    import lightgbm as lgb
    model = lgb.LGBMClassifier(
        boosting_type = boosting_type,
        max_depth = max_depth,
        learning_rate = learning_rate,
        min_split_gain = min_split_gain,
        num_leaves = num_leaves,
        n_estimators = n_estimators,
        min_child_samples = min_child_samples,
        subsample = subsample,
        reg_alpha = reg_alpha,
        reg_lambda = reg_lambda,
        random_state = 42,
        class_weight = 'balanced',
        n_jobs = -1,
        objective = 'binary'
    )
    
    return model

In [22]:
lgbc_low_CIBIL_vf = lgbc_model_builder3(boosting_type = 'gbdt',
        max_depth = 2,
        learning_rate = 0.21925421202433462,
        min_split_gain = 0.10144964926243934,
        num_leaves = 45,
        n_estimators = 319,
        min_child_samples = 221,
        subsample = 0.1959828624191452,
        reg_alpha = 13.828113525346753,
        reg_lambda = 8.84925020519195,
        random_state = 42,
        class_weight = 'balanced',
        n_jobs = -1,
        objective = 'binary'
)

In [40]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, roc_curve, auc
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score
from catboost.utils import get_confusion_matrix, get_roc_curve, get_fnr_curve, get_fpr_curve
from sklearn.metrics import RocCurveDisplay
from sklearn import metrics

segment = 'low_CIBIL'
cleaned_df = pd.read_csv(f'./Wrangled_Data/wrangled_{segment}.csv')

y = 'D3_within12months'
X = cleaned_df.drop(columns = [y, 'loan_account_no'])
y = cleaned_df[y]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 11, train_size = 0.8,
                                                    test_size = 0.2, stratify = y)

object_columns = X.select_dtypes(include=['object','category']).columns

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output = False,drop='first', handle_unknown='ignore').set_output(transform='pandas')

one_hot_encoded_object_X_train = ohe.fit_transform(X_train[object_columns])
one_hot_encoded_object_X_test = ohe.transform(X_test[object_columns])

final_X_train = pd.concat([one_hot_encoded_object_X_train, X_train.select_dtypes('number')], axis=1)
final_X_test = pd.concat([one_hot_encoded_object_X_test, X_test.select_dtypes('number')], axis=1)

X_train = final_X_train
X_test = final_X_test

lgb_model = lgbc_low_CIBIL_vf

lgb_model.fit(X_train, y_train,
               eval_set = [(X_test, y_test)])

[LightGBM] [Info] Number of positive: 1475, number of negative: 14367
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000949 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2517
[LightGBM] [Info] Number of data points in the train set: 15842, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

LGBMClassifier(class_weight='balanced', learning_rate=0.21925421202433462,
               max_depth=2, min_child_samples=221,
               min_split_gain=0.10144964926243934, n_estimators=319, n_jobs=-1,
               num_leaves=45, objective='binary', random_state=42,
               reg_alpha=13.828113525346753, reg_lambda=8.84925020519195,
               subsample=0.1959828624191452)

In [41]:
import pickle
pickle.dump(ohe, open(f'./Refreshed_models/ohe_low_CIBIL.pkl','wb'))
pickle.dump(lgb_model, open(f'./Refreshed_models/lgb_modelr_low_CIBIL.pkl', 'wb'))

In [55]:
segment = 'low_CIBIL'
cleaned_df = pd.read_csv(f'./Wrangled_Data/wrangled_{segment}.csv')
cleaned_df = cleaned_df.drop(columns = ['D3_within12months', 'loan_account_no'])

object_columns = cleaned_df.select_dtypes(include=['object','category']).columns
one_hot_encoded_object_X = ohe.transform(cleaned_df[object_columns])
final_X = pd.concat([one_hot_encoded_object_X, cleaned_df.select_dtypes('number')], axis=1)

y_pred_proba = lgb_model.predict_proba(final_X)[:,1]

cleaned_df['probability'] = y_pred_proba
cleaned_df_low_CIBIL_indexed = cleaned_df.copy()

In [58]:
len(final_X.columns)

44

In [43]:
cleaned_df_low_CIBIL_indexed

,down_payment_amount,disbursal_amount,requested_loan_amount,loan_asset_cost,final_sanctioned_amount,original_emi_amount,age,number_of_installments,total_income,product_description,...,occupation_type,monthly_income,dp_ratio,industry_grp,res_city_grp,res_state_grp,res_resident_status_grp,branch_grp,manufacturer_grp,probability
0,18540.0,74160.0,74160.0,92700,74160.0,3771,41.736264,24,540000.0,TWO-WHEELER,...,SALARIED,45000.000000,0.200000,GROUP_A,GROUP_B,MAHARASHTRA,OTHERWISE,N_WESTERN_MUMBAI,SUZUKI,0.236125
1,15244.0,75000.0,75000.0,90244,75000.0,4846,24.379121,18,750000.0,TWO-WHEELER,...,SELF_EMPLOYED_NON_PROFESSIONAL,62500.000000,0.168920,GROUP_A,GROUP_B,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,N_WESTERN_MUMBAI,GROUP_B,0.234627
2,11400.0,92100.0,92100.0,103500,92100.0,3397,37.587912,36,2000000.0,TWO-WHEELER,...,SELF_EMPLOYED_NON_PROFESSIONAL,166666.666667,0.110145,GROUP_C,GROUP_D,KARNATAKA,OTHERWISE,SOUTH_BANGALORE,GROUP_B,0.592858
3,45800.0,38500.0,38500.0,84300,38500.0,1959,41.815934,24,300000.0,TWO-WHEELER,...,SALARIED,25000.000000,0.543298,GROUP_C,GROUP_C,DELHI,RENTED,HIGH_GROUP,HERO,0.208977
4,4496.0,85000.0,85000.0,89496,85000.0,3132,25.774725,36,420000.0,TWO-WHEELER,...,SALARIED,35000.000000,0.050237,GROUP_C,GROUP_B,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,N_WESTERN_MUMBAI,GROUP_B,0.530895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19798,10800.0,93000.0,93000.0,103800,93000.0,3519,25.469780,36,450000.0,TWO-WHEELER,...,SELF_EMPLOYED_NON_PROFESSIONAL,37500.000000,0.104046,GROUP_A,GROUP_C,UTTAR_PRADESH,RENTED,GHAZIABAD,HERO,0.781631
19799,8000.0,140000.0,140000.0,148000,140000.0,5262,51.244505,36,1050000.0,TWO-WHEELER,...,SELF_EMPLOYED_NON_PROFESSIONAL,87500.000000,0.054054,GROUP_A,GROUP_B,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,N_WESTERN_MUMBAI,SUZUKI,0.497883
19800,20000.0,85000.0,85000.0,105000,85000.0,4401,34.478022,24,354000.0,TWO-WHEELER,...,SELF_EMPLOYED_NON_PROFESSIONAL,29500.000000,0.190476,GROUP_A,GROUP_C,UTTAR_PRADESH,OWNED_BY_PARENT_/SIBLING,GHAZIABAD,SUZUKI,0.432470
19801,11005.0,99045.0,99045.0,110050,99045.0,3653,30.969780,36,420000.0,TWO-WHEELER,...,SALARIED,35000.000000,0.100000,GROUP_C,GROUP_B,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,HARBOUR_MUMBAI,SUZUKI,0.653330


# mid_CIBIL Scoring

In [1]:
#jied
def RF_model_builder2(n_estimators = 100,
                    criterion = 'gini',
                    max_depth = None,
                    min_samples_split = 2,
                    min_samples_leaf = 1,
                    max_features = 'sqrt',
                    max_leaf_nodes = None,
                    min_weight_fraction_leaf = 0.0,
                    random_state = 42
                     ):
    
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(n_estimators = n_estimators, 
                                   criterion = criterion,
                                   max_depth = max_depth, 
                                   min_samples_split = min_samples_split, 
                                   min_samples_leaf = min_samples_leaf, 
                                   max_features = max_features, 
                                   max_leaf_nodes = max_leaf_nodes,
                                   min_weight_fraction_leaf = min_weight_fraction_leaf,
                                   random_state = 42)
    return model

In [4]:
rf_mid_CIBIL_vf = RF_model_builder2(n_estimators = 58,
                    criterion = 'gini',
                    max_depth = 9,
                    min_samples_split = 2,
                    min_samples_leaf = 45,
                    max_features = 'sqrt',
                    max_leaf_nodes = 24,
                    min_weight_fraction_leaf = 0.007818203370596966,
                    random_state = 42)

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, roc_curve, auc
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score
from catboost.utils import get_confusion_matrix, get_roc_curve, get_fnr_curve, get_fpr_curve
from sklearn.metrics import RocCurveDisplay
from sklearn import metrics
import matplotlib.pyplot as plt
#import seaborn as sns

#load df
segment = 'NTC'
cleaned_df = pd.read_csv(f'./Wrangled_Data/wrangled_{segment}.csv')
X_full = cleaned_df.copy()

#remove rows with missing target: D3_within12months
X_full.dropna(axis=0, subset=['D3_within12months'], inplace=True)
y = X_full.D3_within12months
X_full.drop(['D3_within12months','loan_account_no'], axis=1, inplace=True)

from sklearn.model_selection import train_test_split
X_train_full, X_test_full, y_train, y_test = train_test_split(X_full, y, 
                                                               train_size=0.8, test_size=0.2,
                                                               random_state=11, stratify = y)

#select categorical columns
categorical_cols = [cname for cname in X_train_full.columns if
                   X_train_full[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train_full.columns if
                 X_train_full[cname].dtype in ['int64', 'float64']]

#keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()


#preprocess numerical data
numerical_transformer = SimpleImputer(strategy='constant')

#preprocessing categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore',drop='first'))
])

#Bundle preprocessing for num and categ data
preprocessor = ColumnTransformer(
    transformers=[
        ('num',numerical_transformer,numerical_cols),
        ('cat',categorical_transformer,categorical_cols)
    ]
)

rf_model = rf_mid_CIBIL_vf

#Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor',preprocessor),
                     ('model', rf_model)
                     ])

#preprocessing of training data, fit model
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['res_years_at_current_city',
                                                   'age', 'down_payment_amount',
                                                   'total_income',
                                                   'interest_rate',
                                                   'number_of_installments',
                                                   'loan_asset_cost',
                                                   'res_years_at_current_address',
                                                   'years_in_occupation',
                                                   'monthly_income',
                                                   'dp_ratio']),
                                                 ('cat',
                                                  Pipeline(steps=[('imput...
                                                   'occupation_type', 'gender',
                                                   'marital_status',
                                                   'nature_of_business',
                                                   'manufacturer_grp',
                                                   'channel_grp',
                                                   'industry_grp', 'branch_grp',
                                                   'asset_model_grp',
                                                   'res_city_grp',
                                                   'res_state_grp',
                                                   'res_resident_status_grp'])])),
                ('model',
                 RandomForestClassifier(max_depth=9, max_leaf_nodes=24,
                                        min_samples_leaf=45,
                                        min_weight_fraction_leaf=0.007818203370596966,
                                        n_estimators=58, random_state=42))])

In [46]:
import pickle
pickle.dump(ohe, open(f'./Refreshed_models/ohe_mid_CIBIL.pkl','wb'))
pickle.dump(lgb_model, open(f'./Refreshed_models/lgb_modelr_mid_CIBIL.pkl', 'wb'))

In [47]:
segment = 'mid_CIBIL'
cleaned_df = pd.read_csv(f'./Wrangled_Data/wrangled_{segment}.csv')
cleaned_df = cleaned_df.drop(columns = ['D3_within12months', 'loan_account_no'])

object_columns = cleaned_df.select_dtypes(include=['object','category']).columns
one_hot_encoded_object_X = ohe.transform(cleaned_df[object_columns])
final_X = pd.concat([one_hot_encoded_object_X, cleaned_df.select_dtypes('number')], axis=1)

y_pred_proba = lgb_model.predict_proba(final_X)[:,1]

cleaned_df['probability'] = y_pred_proba
cleaned_df_mid_CIBIL_indexed = cleaned_df.copy()

In [48]:
cleaned_df_mid_CIBIL_indexed

,down_payment_amount,disbursal_amount,requested_loan_amount,loan_asset_cost,final_sanctioned_amount,original_emi_amount,age,number_of_installments,total_income,product_description,...,occupation_type,monthly_income,dp_ratio,industry_grp,res_city_grp,res_state_grp,res_resident_status_grp,branch_grp,manufacturer_grp,probability
0,6545.0,118000.0,118000.0,124545,118000.0,4456,56.689560,36,420000.0,TWO-WHEELER,...,SALARIED,35000.000000,0.052551,GROUP_A,GROUP_B,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,S_MUMBAI,GROUP_B,0.400487
1,10210.0,99990.0,99990.0,110200,99990.0,5069,38.348901,24,480000.0,TWO-WHEELER,...,SELF_EMPLOYED_NON_PROFESSIONAL,40000.000000,0.092650,GROUP_C,GROUP_B,MAHARASHTRA,OTHERWISE,HARBOUR_MUMBAI,SUZUKI,0.411463
2,6200.0,91000.0,91000.0,97200,91000.0,3348,36.799451,36,900000.0,TWO-WHEELER,...,SELF_EMPLOYED_NON_PROFESSIONAL,75000.000000,0.063786,GROUP_C,GROUP_A,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,N_WESTERN_MUMBAI,SUZUKI,0.621075
3,5244.0,85000.0,85000.0,90244,85000.0,3130,41.005495,36,1100000.0,TWO-WHEELER,...,SELF_EMPLOYED_NON_PROFESSIONAL,91666.666667,0.058109,GROUP_C,GROUP_B,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,N_WESTERN_MUMBAI,GROUP_B,0.500801
4,42000.0,96000.0,96000.0,138000,96000.0,5344,23.456044,24,360000.0,USED_TWO-WHEELER,...,SALARIED,30000.000000,0.304348,GROUP_A,GROUP_D,KARNATAKA,OWNED_BY_PARENT_/SIBLING,SOUTH_BANGALORE,GROUP_A,0.731311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13080,11600.0,88000.0,88000.0,99600,88000.0,3253,38.804945,36,720000.0,TWO-WHEELER,...,SELF_EMPLOYED_NON_PROFESSIONAL,60000.000000,0.116466,GROUP_A,GROUP_B,MAHARASHTRA,OTHERWISE,S_MUMBAI,SUZUKI,0.296663
13081,5968.0,110000.0,110000.0,115968,110000.0,7215,28.736264,18,432000.0,TWO-WHEELER,...,SALARIED,36000.000000,0.051462,GROUP_A,GROUP_B,MAHARASHTRA,RENTED,EASTERN_MUMBAI,SUZUKI,0.310020
13082,10000.0,106000.0,106000.0,116000,106000.0,5499,44.453297,24,500000.0,TWO-WHEELER,...,SELF_EMPLOYED_NON_PROFESSIONAL,41666.666667,0.086207,GROUP_C,GROUP_C,DELHI,OWNED_BY_PARENT_/SIBLING,HIGH_GROUP,TVS,0.310975
13083,22100.0,73500.0,73500.0,95600,73500.0,3197,31.445055,30,460000.0,TWO-WHEELER,...,SELF_EMPLOYED_NON_PROFESSIONAL,38333.333333,0.231172,GROUP_A,GROUP_D,KARNATAKA,OTHERWISE,HIGH_GROUP,HERO,0.334645


# high_CIBIL scoring

In [9]:
#jied
def RF_model_builder2(n_estimators = 100,
                    criterion = 'gini',
                    max_depth = None,
                    min_samples_split = 2,
                    min_samples_leaf = 1,
                    max_features = 'sqrt',
                    max_leaf_nodes = None,
                    min_weight_fraction_leaf = 0.0,
                    random_state = 42
                     ):
    
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(n_estimators = n_estimators, 
                                   criterion = criterion,
                                   max_depth = max_depth, 
                                   min_samples_split = min_samples_split, 
                                   min_samples_leaf = min_samples_leaf, 
                                   max_features = max_features, 
                                   max_leaf_nodes = max_leaf_nodes,
                                   min_weight_fraction_leaf = min_weight_fraction_leaf,
                                   random_state = 42)
    return model

In [10]:
rf_high_CIBIL_vf =  RF_model_builder2(n_estimators = 493,
                    criterion = 'entropy',
                    max_depth = 5,
                    min_samples_split = 7,
                    min_samples_leaf = 158,
                    max_features = 'log2',
                    max_leaf_nodes = 353,
                    min_weight_fraction_leaf = 0.00038938292050716417,
                    random_state = 42)

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, roc_curve, auc
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score
from catboost.utils import get_confusion_matrix, get_roc_curve, get_fnr_curve, get_fpr_curve
from sklearn.metrics import RocCurveDisplay
from sklearn import metrics
import matplotlib.pyplot as plt
#import seaborn as sns

#load df
segment = 'high_CIBIL'
cleaned_df = pd.read_csv(f'./Wrangled_Data/wrangled_{segment}.csv')
X_full = cleaned_df.copy()

#remove rows with missing target: D3_within12months
X_full.dropna(axis=0, subset=['D3_within12months'], inplace=True)
y = X_full.D3_within12months
X_full.drop(['D3_within12months','loan_account_no'], axis=1, inplace=True)

from sklearn.model_selection import train_test_split
X_train_full, X_test_full, y_train, y_test = train_test_split(X_full, y, 
                                                               train_size=0.8, test_size=0.2,
                                                               random_state=11, stratify = y)

#select categorical columns
categorical_cols = [cname for cname in X_train_full.columns if
                   X_train_full[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train_full.columns if
                 X_train_full[cname].dtype in ['int64', 'float64']]

#keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()


#preprocess numerical data
numerical_transformer = SimpleImputer(strategy='constant')

#preprocessing categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore',drop='first'))
])

#Bundle preprocessing for num and categ data
preprocessor = ColumnTransformer(
    transformers=[
        ('num',numerical_transformer,numerical_cols),
        ('cat',categorical_transformer,categorical_cols)
    ]
)

rf_model = rf_high_CIBIL_vf

#Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor',preprocessor),
                     ('model', rf_model)
                     ])

#preprocessing of training data, fit model
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['down_payment_amount',
                                                   'disbursal_amount',
                                                   'requested_loan_amount',
                                                   'loan_asset_cost',
                                                   'final_sanctioned_amount',
                                                   'original_emi_amount', 'age',
                                                   'number_of_installments',
                                                   'total_income',
                                                   'monthly_income',
                                                   'dp_ratio']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   Si...
                                                   'occupation_type',
                                                   'industry_grp',
                                                   'res_city_grp',
                                                   'res_state_grp',
                                                   'res_resident_status_grp',
                                                   'branch_grp',
                                                   'manufacturer_grp'])])),
                ('model',
                 RandomForestClassifier(criterion='entropy', max_depth=5,
                                        max_features='log2', max_leaf_nodes=353,
                                        min_samples_leaf=158,
                                        min_samples_split=7,
                                        min_weight_fraction_leaf=0.00038938292050716417,
                                        n_estimators=493, random_state=42))])

In [13]:
import pickle
pickle.dump(clf, open(f'./Refreshed_models/rf_modelr_high_CIBIL_corrected.pkl', 'wb'))

In [14]:
y_pred_proba = clf.predict_proba(X_full)[:,1]

cleaned_df['probability'] = y_pred_proba
cleaned_df_high_CIBIL_indexed_chq = cleaned_df.copy()

In [54]:
cleaned_df_high_CIBIL_indexed_chq #wrongly saved

,down_payment_amount,disbursal_amount,requested_loan_amount,loan_asset_cost,final_sanctioned_amount,original_emi_amount,age,number_of_installments,total_income,product_description,...,monthly_income,dp_ratio,D3_within12months,industry_grp,res_city_grp,res_state_grp,res_resident_status_grp,branch_grp,manufacturer_grp,probability
0,18200.0,85000.0,85000.0,103200,85000.0,3118,41.695055,36,900000.0,TWO-WHEELER,...,75000.000000,0.176357,0,GROUP_C,GROUP_A,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,N_WESTERN_MUMBAI,SUZUKI,0.281863
1,10100.0,40400.0,40400.0,50500,40400.0,3855,24.829670,12,250000.0,USED_TWO-WHEELER,...,20833.333333,0.200000,0,GROUP_A,GROUP_C,DELHI,RENTED,WEST_DELHI,TVS,0.893492
2,15244.0,75000.0,75000.0,90244,75000.0,2752,27.777473,36,276000.0,TWO-WHEELER,...,23000.000000,0.168920,0,GROUP_A,GROUP_A,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,N_WESTERN_MUMBAI,GROUP_B,0.125963
3,11815.0,89500.0,89500.0,101315,89500.0,3364,34.021978,36,420000.0,TWO-WHEELER,...,35000.000000,0.116616,0,GROUP_A,GROUP_A,MAHARASHTRA,OTHERWISE,LOW_GROUP,GROUP_B,0.246760
4,11500.0,63000.0,63000.0,74500,63000.0,4135,34.722527,18,336000.0,TWO-WHEELER,...,28000.000000,0.154362,0,GROUP_A,GROUP_C,DELHI,OWNED_BY_PARENT_/SIBLING,WEST_DELHI,BAJAJ_AUTO,0.356569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22678,27000.0,47000.0,47000.0,74000,47000.0,2547,43.510989,24,360000.0,USED_TWO-WHEELER,...,30000.000000,0.364865,0,GROUP_A,GROUP_B,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,HARBOUR_MUMBAI,TVS,0.082053
22679,15513.0,71000.0,71000.0,86513,71000.0,3080,49.519231,30,420000.0,TWO-WHEELER,...,35000.000000,0.179314,0,GROUP_A,GROUP_D,KARNATAKA,RENTED,HIGH_GROUP,HERO,0.523961
22680,76.0,98000.0,98000.0,98076,98000.0,3691,39.549451,36,480000.0,TWO-WHEELER,...,40000.000000,0.000775,0,GROUP_A,GROUP_B,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,HARBOUR_MUMBAI,TVS,0.557007
22681,8100.0,116500.0,116500.0,124600,116500.0,4404,26.983516,36,540000.0,TWO-WHEELER,...,45000.000000,0.065008,0,GROUP_A,GROUP_D,KARNATAKA,RENTED,SOUTH_BANGALORE,TVS,0.475097


In [15]:
cleaned_df_high_CIBIL_indexed_chq #corrected scores

,down_payment_amount,disbursal_amount,requested_loan_amount,loan_asset_cost,final_sanctioned_amount,original_emi_amount,age,number_of_installments,total_income,product_description,...,monthly_income,dp_ratio,D3_within12months,industry_grp,res_city_grp,res_state_grp,res_resident_status_grp,branch_grp,manufacturer_grp,probability
0,18200.0,85000.0,85000.0,103200,85000.0,3118,41.695055,36,900000.0,TWO-WHEELER,...,75000.000000,0.176357,0,GROUP_C,GROUP_A,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,N_WESTERN_MUMBAI,SUZUKI,0.018886
1,10100.0,40400.0,40400.0,50500,40400.0,3855,24.829670,12,250000.0,USED_TWO-WHEELER,...,20833.333333,0.200000,0,GROUP_A,GROUP_C,DELHI,RENTED,WEST_DELHI,TVS,0.053189
2,15244.0,75000.0,75000.0,90244,75000.0,2752,27.777473,36,276000.0,TWO-WHEELER,...,23000.000000,0.168920,0,GROUP_A,GROUP_A,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,N_WESTERN_MUMBAI,GROUP_B,0.023414
3,11815.0,89500.0,89500.0,101315,89500.0,3364,34.021978,36,420000.0,TWO-WHEELER,...,35000.000000,0.116616,0,GROUP_A,GROUP_A,MAHARASHTRA,OTHERWISE,LOW_GROUP,GROUP_B,0.032635
4,11500.0,63000.0,63000.0,74500,63000.0,4135,34.722527,18,336000.0,TWO-WHEELER,...,28000.000000,0.154362,0,GROUP_A,GROUP_C,DELHI,OWNED_BY_PARENT_/SIBLING,WEST_DELHI,BAJAJ_AUTO,0.036143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22678,27000.0,47000.0,47000.0,74000,47000.0,2547,43.510989,24,360000.0,USED_TWO-WHEELER,...,30000.000000,0.364865,0,GROUP_A,GROUP_B,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,HARBOUR_MUMBAI,TVS,0.019143
22679,15513.0,71000.0,71000.0,86513,71000.0,3080,49.519231,30,420000.0,TWO-WHEELER,...,35000.000000,0.179314,0,GROUP_A,GROUP_D,KARNATAKA,RENTED,HIGH_GROUP,HERO,0.038862
22680,76.0,98000.0,98000.0,98076,98000.0,3691,39.549451,36,480000.0,TWO-WHEELER,...,40000.000000,0.000775,0,GROUP_A,GROUP_B,MAHARASHTRA,OWNED_BY_PARENT_/SIBLING,HARBOUR_MUMBAI,TVS,0.045640
22681,8100.0,116500.0,116500.0,124600,116500.0,4404,26.983516,36,540000.0,TWO-WHEELER,...,45000.000000,0.065008,0,GROUP_A,GROUP_D,KARNATAKA,RENTED,SOUTH_BANGALORE,TVS,0.064076


In [91]:
y_pred_proba.max()

0.09212157741835753

In [32]:
print(cleaned_df_high_CIBIL.shape)
print(cleaned_df_mid_CIBIL.shape)
print(cleaned_df_low_CIBIL.shape)
print(cleaned_df_NTC.shape)

(22683, 28)
(13085, 28)
(19803, 28)
(31518, 28)


In [33]:
22683+13085+19803+31518

87089

In [ ]:
scored_df_full = pd.concat([df1])

In [35]:
cleaned_df_NTC.columns

Index(['dp_ratio', 'down_payment_amount', 'disbursal_amount',
       'requested_loan_amount', 'loan_asset_cost', 'final_sanctioned_amount',
       'original_emi_amount', 'age', 'number_of_installments', 'total_income',
       'product_description', 'years_other_add', 'tat', 'marital_status',
       'monthly_income', 'occupation_type', 'D3_within12months',
       'deviation_cnt_grp', 'verification_waived_grp', 'PerVeri_TelVer_grp',
       'ResiVeri_FieldVer_grp', 'industry_grp', 'res_city_grp',
       'res_state_grp', 'res_resident_status_grp', 'branch_grp',
       'manufacturer_grp', 'probability'],
      dtype='object')

In [78]:
cleaned_df_high_CIBIL.probability

0        0.019051
3        0.050064
4        0.021323
7        0.031145
11       0.035112
           ...   
87073    0.018366
87074    0.038080
87078    0.044395
87083    0.062914
87087    0.038059
Name: probability, Length: 22683, dtype: float64

In [77]:
cleaned_df_high_CIBIL_indexed.probability

0        0.019051
3        0.050064
4        0.021323
7        0.031145
11       0.035112
           ...   
87073    0.018366
87074    0.038080
87078    0.044395
87083    0.062914
87087    0.038059
Name: probability, Length: 22683, dtype: float64

In [79]:
df_scored_indexed_full = pd.concat([cleaned_df_NTC_indexed, cleaned_df_low_CIBIL_indexed,
                              cleaned_df_mid_CIBIL_indexed,cleaned_df_high_CIBIL_indexed])

In [82]:
df_scored_indexed_full.to_csv('scored_data_with_account_no.csv')